In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pylab
import glob 

In [88]:
### Reading data from files 
dataDir =r"C:/Ramya Dhatri/Intro To Data Science/Project/Dock/Output/"
files = glob.glob(dataDir + "output_*.csv")
data = pd.DataFrame()
dock_accuracy = pd.DataFrame(dict(Dock_ID=[], True_Predictions=[], False_Predictions=[], Total_Predictions=[]), 
                             columns=['Dock_ID', 'True_Predictions', 'False_Predictions', 'Total_Predictions'], dtype=int)
list_ = []
dock_count = 0
for file_ in files:
    tempData = pd.read_csv(file_,index_col=None, sep='\t', header=0)
    true_pred = 0
    false_pred = 0 
    total_pred = len(tempData)
    dock_id = tempData.DockID.values[0]
    for index, row in tempData.iterrows():
        if (row['Predicted_Bin'] == row['Expected_Bin']):
            true_pred += 1
        else:
            false_pred += 1
    dock_accuracy.loc[dock_count] = [dock_id, true_pred, false_pred, int(total_pred)]
    list_.append(tempData)
    dock_count += 1
data = pd.concat(list_)
data = data.drop(data.columns[[0]], axis=1)

In [89]:
dock_accuracy

,Dock_ID,True_Predictions,False_Predictions,Total_Predictions
0,152,1007,74,1081
1,322,946,135,1081
2,325,944,137,1081
3,368,1010,71,1081
4,379,983,98,1081
5,380,986,95,1081
6,390,975,106,1081
7,490,959,122,1081
8,503,932,149,1081
9,82,1006,75,1081


In [3]:
from sklearn.metrics import roc_curve, auc

def plotUnivariateROC(preds, truth, label_string):
    fpr, tpr, thresholds = roc_curve(truth, preds)
    roc_auc = auc(fpr, tpr)

    #chooses a random color for plotting
    c = (np.random.rand(), np.random.rand(), np.random.rand())

    #create a plot and set some options
    plt.plot(fpr, tpr, color = c, label = label_string + ' (AUC = %0.3f)' % roc_auc)
    

    plt.plot([0, 1], [0, 1], 'k--')
    #plt.xlim([0.0, 1.0])
    #plt.ylim([0.0, 1.0])
    plt.xlabel('FPR')
    plt.ylabel('TPR')
    plt.title('ROC')
    plt.legend(loc="lower right")
    
    return roc_auc

In [ ]:
fig = plt.figure(figsize = (12, 6))
ax = plt.subplot(111)

auc_dict = {}
#for index in range(len(data)):
auc_dict[index] = plotUnivariateROC(data['Predicted_Bin'], data['Expected_Bin'], 'Time Series')


# Put a legend below current axis
box = ax.get_position()
ax.set_position([box.x0, box.y0 + box.height * 0.0 , box.width, box.height * 1])
ax.legend(loc = 'upper center', bbox_to_anchor = (0.5, -0.15), fancybox = True, 
              shadow = True, ncol = 4, prop = {'size':10})
plt.show()

In [90]:
from sklearn import metrics

ts_fpr, ts_tpr, ts_threshold = metrics.roc_curve (data['Expected_Bin'], data['Predicted_Bin'])
ts_auc = metrics.auc(ts_fpr, ts_tpr)
ts_accuracy = metrics.accuracy_score(data['Expected_Bin'], data['Predicted_Bin'])
ts_recall = metrics.recall_score(data['Expected_Bin'], data['Predicted_Bin'])
ts_precision = metrics.precision_score(data['Expected_Bin'], data['Predicted_Bin'])
ts_f1 = metrics.f1_score(data['Expected_Bin'], data['Predicted_Bin'])
print('ts_accuracy = {accu:5.3f}, ts_auc = {auc:5.3f}, ts_recall = {rec:5.3f}, ts_precision = {prec:5.3f}, ts_f1 = {f1:5.3f}'.format(
        accu = ts_accuracy, auc = ts_auc, rec = ts_recall, prec = ts_precision, f1 = ts_f1))


ts_accuracy = 0.902, ts_auc = 0.888, ts_recall = 0.844, ts_precision = 0.867, ts_f1 = 0.856


In [ ]:
plt.plot(ts_fpr,ts_tpr, label = 'ts: AUC = {auc:5.3f}'.format(auc = ts_auc))
plt.show()